<a href="https://colab.research.google.com/github/abirinpajamas/adhd-prediction-ml-model/blob/main/hyperactiv_adhd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

HYPERAKTIV is a public dataset containing health, activity, andheart rate data from adult patients diagnosed with attention deficit hyperactivity disorder, better known as ADHD. The dataset consists of data collected from 51 patients with ADHD and 52 clinical-controls. In addition to the activity and heart rate data, also includes a series of patient attributes such as their age, sex, and information about their mental state, as well as output data from a computerized neuropsychological test.


In [2]:
df_f=pd.read_csv("features.csv",sep=";")
pd.set_option('display.max_rows', None)


df_f.head()


,ID,ACC__variance_larger_than_standard_deviation,ACC__has_duplicate_max,ACC__has_duplicate_min,ACC__has_duplicate,ACC__sum_values,ACC__abs_energy,ACC__mean_abs_change,ACC__mean_change,ACC__mean_second_derivative_central,...,ACC__permutation_entropy__dimension_5__tau_1,ACC__permutation_entropy__dimension_6__tau_1,ACC__permutation_entropy__dimension_7__tau_1,ACC__query_similarity_count__query_None__threshold_0.0,"ACC__matrix_profile__feature_""min""__threshold_0.98","ACC__matrix_profile__feature_""max""__threshold_0.98","ACC__matrix_profile__feature_""mean""__threshold_0.98","ACC__matrix_profile__feature_""median""__threshold_0.98","ACC__matrix_profile__feature_""25""__threshold_0.98","ACC__matrix_profile__feature_""75""__threshold_0.98"
0,1,1.0,0.0,1.0,1.0,433530.0,1.228179e+08,34.574870,0.006254,-0.015752,...,3.120897,4.077141,4.880321,NaN,3.495156,15.586971,11.632054,12.013496,10.744060,13.176179
1,2,1.0,1.0,1.0,1.0,1572295.0,1.010897e+09,103.330694,0.000000,-0.002792,...,3.883058,5.149344,6.278021,NaN,3.054584,15.231546,9.369168,9.348224,8.285911,10.382951
2,3,1.0,0.0,1.0,1.0,5232087.0,6.869091e+09,251.845947,0.000000,-0.049439,...,4.364522,5.866384,7.260060,NaN,3.947638,19.595918,13.502457,13.651750,12.254941,14.778953
3,5,1.0,0.0,1.0,1.0,508325.0,1.018261e+08,46.477767,0.000000,-0.004494,...,4.096368,5.464033,6.680095,NaN,3.695875,14.341713,11.853373,12.124791,11.190192,12.813456
4,7,1.0,0.0,1.0,1.0,641357.0,1.545783e+08,34.675614,0.000000,-0.015340,...,3.415288,4.490762,5.442471,NaN,3.500962,22.090722,14.492647,14.989601,12.924897,16.403917


In [3]:
df=pd.read_csv("patient_info.csv",sep=";")
pd.set_option('display.max_columns', None)


df.head()


,ID,SEX,AGE,ACC,ACC_TIME,ACC_DAYS,HRV,HRV_TIME,HRV_HOURS,CPT_II,ADHD,ADD,BIPOLAR,UNIPOLAR,ANXIETY,SUBSTANCE,OTHER,CT,MDQ_POS,WURS,ASRS,MADRS,HADS_A,HADS_D,MED,MED_Antidepr,MED_Moodstab,MED_Antipsych,MED_Anxiety_Benzo,MED_Sleep,MED_Analgesics_Opioids,MED_Stimulants,filter_$
0,1,0,3,1,16:00:00,6.0,1,11:00:00,21.0,0,1,0,1,0,1,0,1,1,1,54.0,53.0,26.0,12.0,2.0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2,0,4,1,10:54:00,6.8,0,NaN,NaN,1,1,0,0,1,0,0,1,1,1,83.0,62.0,11.0,2.0,7.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,3,1,2,1,15:28:00,7.2,1,15:25:00,21.0,1,1,0,1,0,0,1,0,0,0,72.0,60.0,6.0,8.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4,1,3,0,NaN,NaN,1,16:55:00,22.0,1,1,0,1,0,1,1,0,0,9,82.0,46.0,19.0,12.0,6.0,1,1.0,NaN,NaN,1.0,NaN,1.0,NaN,0
4,5,1,1,1,14:24:00,5.9,1,16:00:00,12.0,1,1,1,1,0,1,0,1,1,1,60.0,56.0,28.0,10.0,5.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [4]:
dcols= [
        'ACC',	'ACC_TIME',
        'ACC_DAYS',	'HRV',
        'HRV_TIME',	'HRV_HOURS',
        'CPT_II',
       'MED',	'MED_Antidepr',
       'MED_Moodstab',	'MED_Antipsych',
       'MED_Anxiety_Benzo',	'MED_Sleep',
       'MED_Analgesics_Opioids',
       'MED_Stimulants'

]
df.drop(columns=dcols,inplace=True)
pd.set_option('display.max_columns', None)
df.head()

,ID,SEX,AGE,ADHD,ADD,BIPOLAR,UNIPOLAR,ANXIETY,SUBSTANCE,OTHER,CT,MDQ_POS,WURS,ASRS,MADRS,HADS_A,HADS_D,filter_$
0,1,0,3,1,0,1,0,1,0,1,1,1,54.0,53.0,26.0,12.0,2.0,1
1,2,0,4,1,0,0,1,0,0,1,1,1,83.0,62.0,11.0,2.0,7.0,1
2,3,1,2,1,0,1,0,0,1,0,0,0,72.0,60.0,6.0,8.0,0.0,1
3,4,1,3,1,0,1,0,1,1,0,0,9,82.0,46.0,19.0,12.0,6.0,0
4,5,1,1,1,1,1,0,1,0,1,1,1,60.0,56.0,28.0,10.0,5.0,1


In [5]:
merged_df = pd.merge(df, df_f, on='ID', how='inner')
merged_df.head()

,ID,SEX,AGE,ADHD,ADD,BIPOLAR,UNIPOLAR,ANXIETY,SUBSTANCE,OTHER,CT,MDQ_POS,WURS,ASRS,MADRS,HADS_A,HADS_D,filter_$,ACC__variance_larger_than_standard_deviation,ACC__has_duplicate_max,ACC__has_duplicate_min,ACC__has_duplicate,ACC__sum_values,ACC__abs_energy,ACC__mean_abs_change,ACC__mean_change,ACC__mean_second_derivative_central,ACC__median,ACC__mean,ACC__length,ACC__standard_deviation,ACC__variation_coefficient,ACC__variance,ACC__skewness,ACC__kurtosis,ACC__root_mean_square,ACC__absolute_sum_of_changes,ACC__longest_strike_below_mean,ACC__longest_strike_above_mean,ACC__count_above_mean,ACC__count_below_mean,ACC__last_location_of_maximum,ACC__first_location_of_maximum,ACC__last_location_of_minimum,ACC__first_location_of_minimum,ACC__percentage_of_reoccurring_values_to_all_values,ACC__percentage_of_reoccurring_datapoints_to_all_datapoints,ACC__sum_of_reoccurring_values,ACC__sum_of_reoccurring_data_points,ACC__ratio_value_number_to_time_series_length,ACC__sample_entropy,ACC__maximum,ACC__minimum,ACC__benford_correlation,ACC__time_reversal_asymmetry_statistic__lag_1,ACC__time_reversal_asymmetry_statistic__lag_2,ACC__time_reversal_asymmetry_statistic__lag_3,ACC__c3__lag_1,ACC__c3__lag_2,ACC__c3__lag_3,ACC__cid_ce__normalize_True,ACC__cid_ce__normalize_False,ACC__symmetry_looking__r_0.0,ACC__symmetry_looking__r_0.05,ACC__symmetry_looking__r_0.1,ACC__symmetry_looking__r_0.15000000000000002,ACC__symmetry_looking__r_0.2,ACC__symmetry_looking__r_0.25,ACC__symmetry_looking__r_0.30000000000000004,ACC__symmetry_looking__r_0.35000000000000003,ACC__symmetry_looking__r_0.4,ACC__symmetry_looking__r_0.45,ACC__symmetry_looking__r_0.5,ACC__symmetry_looking__r_0.55,ACC__symmetry_looking__r_0.6000000000000001,ACC__symmetry_looking__r_0.65,ACC__symmetry_looking__r_0.7000000000000001,ACC__symmetry_looking__r_0.75,ACC__symmetry_looking__r_0.8,ACC__symmetry_looking__r_0.8500000000000001,ACC__symmetry_looking__r_0.9,ACC__symmetry_looking__r_0.9500000000000001,ACC__large_standard_deviation__r_0.05,ACC__large_standard_deviation__r_0.1,ACC__large_standard_deviation__r_0.15000000000000002,ACC__large_standard_deviation__r_0.2,ACC__large_standard_deviation__r_0.25,ACC__large_standard_deviation__r_0.30000000000000004,ACC__large_standard_deviation__r_0.35000000000000003,ACC__large_standard_deviation__r_0.4,ACC__large_standard_deviation__r_0.45,ACC__large_standard_deviation__r_0.5,ACC__large_standard_deviation__r_0.55,ACC__large_standard_deviation__r_0.6000000000000001,ACC__large_standard_deviation__r_0.65,ACC__large_standard_deviation__r_0.7000000000000001,ACC__large_standard_deviation__r_0.75,ACC__large_standard_deviation__r_0.8,ACC__large_standard_deviation__r_0.8500000000000001,ACC__large_standard_deviation__r_0.9,ACC__large_standard_deviation__r_0.9500000000000001,ACC__quantile__q_0.1,ACC__quantile__q_0.2,ACC__quantile__q_0.3,ACC__quantile__q_0.4,ACC__quantile__q_0.6,ACC__quantile__q_0.7,ACC__quantile__q_0.8,ACC__quantile__q_0.9,ACC__autocorrelation__lag_0,ACC__autocorrelation__lag_1,ACC__autocorrelation__lag_2,ACC__autocorrelation__lag_3,ACC__autocorrelation__lag_4,ACC__autocorrelation__lag_5,ACC__autocorrelation__lag_6,ACC__autocorrelation__lag_7,ACC__autocorrelation__lag_8,ACC__autocorrelation__lag_9,"ACC__agg_autocorrelation__f_agg_""mean""__maxlag_40","ACC__agg_autocorrelation__f_agg_""median""__maxlag_40","ACC__agg_autocorrelation__f_agg_""var""__maxlag_40",ACC__partial_autocorrelation__lag_0,ACC__partial_autocorrelation__lag_1,ACC__partial_autocorrelation__lag_2,ACC__partial_autocorrelation__lag_3,ACC__partial_autocorrelation__lag_4,ACC__partial_autocorrelation__lag_5,ACC__partial_autocorrelation__lag_6,ACC__partial_autocorrelation__lag_7,ACC__partial_autocorrelation__lag_8,ACC__partial_autocorrelation__lag_9,ACC__number_cwt_peaks__n_1,ACC__number_cwt_peaks__n_5,ACC__number_peaks__n_1,ACC__number_peaks__n_3,ACC__number_peaks__n_5,ACC__number_peaks__n_10,ACC__number_peaks__n_50,ACC__binned_entropy__max_bins_10,ACC__index_mass_quantile__q_0.1,ACC__index_m

In [6]:
len(merged_df)

85

In [7]:
cdf=pd.read_csv('CPT_II_ConnersContinuousPerformanceTest.csv',sep=";")
cdf.head()

,ID,Assessment Status,Assessment Duration,Type,LastTrial,Trial1,Response1,Trial2,Response2,Trial3,Response3,Trial4,Response4,Trial5,Response5,Trial6,Response6,Trial7,Response7,Trial8,Response8,Trial9,Response9,Trial10,Response10,Trial11,Response11,Trial12,Response12,Trial13,Response13,Trial14,Response14,Trial15,Response15,Trial16,Response16,Trial17,Response17,Trial18,Response18,Trial19,Response19,Trial20,Response20,Trial21,Response21,Trial22,Response22,Trial23,Response23,Trial24,Response24,Trial25,Response25,Trial26,Response26,Trial27,Response27,Trial28,Response28,Trial29,Response29,Trial30,Response30,Trial31,Response31,Trial32,Response32,Trial33,Response33,Trial34,Response34,Trial35,Response35,Trial36,Response36,Trial37,Response37,Trial38,Response38,Trial39,Response39,Trial40,Response40,Trial41,Response41,Trial42,Response42,Trial43,Response43,Trial44,Response44,Trial45,Response45,Trial46,Response46,Trial47,Response47,Trial48,Response48,Trial49,Response49,Trial50,Response50,Trial51,Response51,Trial52,Response52,Trial53,Response53,Trial54,Response54,Trial55,Response55,Trial56,Response56,Trial57,Response57,Trial58,Response58,Trial59,Response59,Trial60,Response60,Trial61,Response61,Trial62,Response62,Trial63,Response63,Trial64,Response64,Trial65,Response65,Trial66,Response66,Trial67,Response67,Trial68,Response68,Trial69,Response69,Trial70,Response70,Trial71,Response71,Trial72,Response72,Trial73,Response73,Trial74,Response74,Trial75,Response75,Trial76,Response76,Trial77,Response77,Trial78,Response78,Trial79,Response79,Trial80,Response80,Trial81,Response81,Trial82,Response82,Trial83,Response83,Trial84,Response84,Trial85,Response85,Trial86,Response86,Trial87,Response87,Trial88,Response88,Trial89,Response89,Trial90,Response90,Trial91,Response91,Trial92,Response92,Trial93,Response93,Trial94,Response94,Trial95,Response95,Trial96,Response96,Trial97,Response97,Trial98,Response98,Trial99,Response99,Trial100,Response100,Trial101,Response101,Trial102,Response102,Trial103,Response103,Trial104,Response104,Trial105,Response105,Trial106,Response106,Trial107,Response107,Trial108,Response108,Trial109,Response109,Trial110,Response110,Trial111,Response111,Trial112,Response112,Trial113,Response113,Trial114,Response114,Trial115,Response115,Trial116,Response116,Trial117,Response117,Trial118,Response118,Trial119,Response119,Trial120,Response120,Trial121,Response121,Trial122,Response122,Trial123,Response123,Trial124,Response124,Trial125,Response125,Trial126,Response126,Trial127,Response127,Trial128,Response128,Trial129,Response129,Trial130,Response130,Trial131,Response131,Trial132,Response132,Trial133,Response133,Trial134,Response134,Trial135,Response135,Trial136,Response136,Trial137,Response137,Trial138,Response138,Trial139,Response139,Trial140,Response140,Trial141,Response141,Trial142,Response142,Trial143,Response143,Trial144,Response144,Trial145,Response145,Trial146,Response146,Trial147,Response147,Trial148,Response148,Trial149,Response149,Trial150,Response150,Trial151,Response151,Trial152,Response152,Trial153,Response153,Trial154,Response154,Trial155,Response155,Trial156,Response156,Trial157,Response157,Trial158,Response158,Trial159,Response159,Trial160,Response160,Trial161,Response161,Trial162,Response162,Trial163,Response163,Trial164,Response164,Trial165,Response165,Trial166,Response166,Trial167,Response167,Trial168,Response168,Trial169,Response169,Trial170,Response170,Trial171,Response171,Trial172,Response172,Trial173,Response173,Trial174,Response174,Trial175,Response175,Trial176,Response176,Trial177,Response177,Trial178,Response178,Trial179,Response179,Trial180,Response180,Trial181,Response181,Trial182,Response182,Trial183,Response183,Trial184,Response184,Trial185,Response185,Trial186,Response186,Trial187,Response187,Trial188,Response188,Trial189,Response189,Trial190,Response190,Trial191,Response191,Trial192,Response192,Trial193,Response193,Trial194,Response194,Trial195,Response195,Trial196,Response196,Trial197,Response197,Trial198,Response198,T

In [8]:
cpt_columns = [
    'ID',
    'Raw Score Omissions',
    'Raw Score Commissions',
    'Raw Score HitRT',
    'Raw Score HitSE',
    'Raw Score VarSE',
    'Raw Score DPrime',
    'Raw Score Beta',
    'Adhd Confidence Index',
    'Neuro Confidence Index',
    'Old Overall Index'
]


cdf=cdf[cpt_columns]
cdf.head()

,ID,Raw Score Omissions,Raw Score Commissions,Raw Score HitRT,Raw Score HitSE,Raw Score VarSE,Raw Score DPrime,Raw Score Beta,Adhd Confidence Index,Neuro Confidence Index,Old Overall Index
0,2,5,28,383.820740,7.099574,11.213081,0.045588,0.382657,61.145496,58.749630,10.070000
1,3,11,28,325.208466,5.512853,7.517670,0.197340,0.501448,86.871597,61.088688,4.440000
2,4,15,23,474.340973,8.852735,7.212327,0.165447,0.815376,99.900002,86.131813,18.340000
3,5,10,32,350.914764,11.575007,43.372417,0.282014,0.497768,99.900002,99.900002,8.950001
4,7,1,11,384.021728,5.002260,5.363655,0.624464,0.752431,32.291084,24.225414,0.000000


In [9]:
final_db=pd.merge(merged_df,cdf,on='ID',how='inner')
final_db.head()

,ID,SEX,AGE,ADHD,ADD,BIPOLAR,UNIPOLAR,ANXIETY,SUBSTANCE,OTHER,CT,MDQ_POS,WURS,ASRS,MADRS,HADS_A,HADS_D,filter_$,ACC__variance_larger_than_standard_deviation,ACC__has_duplicate_max,ACC__has_duplicate_min,ACC__has_duplicate,ACC__sum_values,ACC__abs_energy,ACC__mean_abs_change,ACC__mean_change,ACC__mean_second_derivative_central,ACC__median,ACC__mean,ACC__length,ACC__standard_deviation,ACC__variation_coefficient,ACC__variance,ACC__skewness,ACC__kurtosis,ACC__root_mean_square,ACC__absolute_sum_of_changes,ACC__longest_strike_below_mean,ACC__longest_strike_above_mean,ACC__count_above_mean,ACC__count_below_mean,ACC__last_location_of_maximum,ACC__first_location_of_maximum,ACC__last_location_of_minimum,ACC__first_location_of_minimum,ACC__percentage_of_reoccurring_values_to_all_values,ACC__percentage_of_reoccurring_datapoints_to_all_datapoints,ACC__sum_of_reoccurring_values,ACC__sum_of_reoccurring_data_points,ACC__ratio_value_number_to_time_series_length,ACC__sample_entropy,ACC__maximum,ACC__minimum,ACC__benford_correlation,ACC__time_reversal_asymmetry_statistic__lag_1,ACC__time_reversal_asymmetry_statistic__lag_2,ACC__time_reversal_asymmetry_statistic__lag_3,ACC__c3__lag_1,ACC__c3__lag_2,ACC__c3__lag_3,ACC__cid_ce__normalize_True,ACC__cid_ce__normalize_False,ACC__symmetry_looking__r_0.0,ACC__symmetry_looking__r_0.05,ACC__symmetry_looking__r_0.1,ACC__symmetry_looking__r_0.15000000000000002,ACC__symmetry_looking__r_0.2,ACC__symmetry_looking__r_0.25,ACC__symmetry_looking__r_0.30000000000000004,ACC__symmetry_looking__r_0.35000000000000003,ACC__symmetry_looking__r_0.4,ACC__symmetry_looking__r_0.45,ACC__symmetry_looking__r_0.5,ACC__symmetry_looking__r_0.55,ACC__symmetry_looking__r_0.6000000000000001,ACC__symmetry_looking__r_0.65,ACC__symmetry_looking__r_0.7000000000000001,ACC__symmetry_looking__r_0.75,ACC__symmetry_looking__r_0.8,ACC__symmetry_looking__r_0.8500000000000001,ACC__symmetry_looking__r_0.9,ACC__symmetry_looking__r_0.9500000000000001,ACC__large_standard_deviation__r_0.05,ACC__large_standard_deviation__r_0.1,ACC__large_standard_deviation__r_0.15000000000000002,ACC__large_standard_deviation__r_0.2,ACC__large_standard_deviation__r_0.25,ACC__large_standard_deviation__r_0.30000000000000004,ACC__large_standard_deviation__r_0.35000000000000003,ACC__large_standard_deviation__r_0.4,ACC__large_standard_deviation__r_0.45,ACC__large_standard_deviation__r_0.5,ACC__large_standard_deviation__r_0.55,ACC__large_standard_deviation__r_0.6000000000000001,ACC__large_standard_deviation__r_0.65,ACC__large_standard_deviation__r_0.7000000000000001,ACC__large_standard_deviation__r_0.75,ACC__large_standard_deviation__r_0.8,ACC__large_standard_deviation__r_0.8500000000000001,ACC__large_standard_deviation__r_0.9,ACC__large_standard_deviation__r_0.9500000000000001,ACC__quantile__q_0.1,ACC__quantile__q_0.2,ACC__quantile__q_0.3,ACC__quantile__q_0.4,ACC__quantile__q_0.6,ACC__quantile__q_0.7,ACC__quantile__q_0.8,ACC__quantile__q_0.9,ACC__autocorrelation__lag_0,ACC__autocorrelation__lag_1,ACC__autocorrelation__lag_2,ACC__autocorrelation__lag_3,ACC__autocorrelation__lag_4,ACC__autocorrelation__lag_5,ACC__autocorrelation__lag_6,ACC__autocorrelation__lag_7,ACC__autocorrelation__lag_8,ACC__autocorrelation__lag_9,"ACC__agg_autocorrelation__f_agg_""mean""__maxlag_40","ACC__agg_autocorrelation__f_agg_""median""__maxlag_40","ACC__agg_autocorrelation__f_agg_""var""__maxlag_40",ACC__partial_autocorrelation__lag_0,ACC__partial_autocorrelation__lag_1,ACC__partial_autocorrelation__lag_2,ACC__partial_autocorrelation__lag_3,ACC__partial_autocorrelation__lag_4,ACC__partial_autocorrelation__lag_5,ACC__partial_autocorrelation__lag_6,ACC__partial_autocorrelation__lag_7,ACC__partial_autocorrelation__lag_8,ACC__partial_autocorrelation__lag_9,ACC__number_cwt_peaks__n_1,ACC__number_cwt_peaks__n_5,ACC__number_peaks__n_1,ACC__number_peaks__n_3,ACC__number_peaks__n_5,ACC__number_peaks__n_10,ACC__number_peaks__n_50,ACC__binned_entropy__max_bins_10,ACC__index_mass_quantile__q_0.1,ACC__index_m

In [10]:
x=final_db.drop(columns=['ID','ADHD'])
y=final_db['ADHD']

In [11]:
columns_drop = [
    'ACC__friedrich_coefficients__coeff_0__m_3__r_30',
    'ACC__friedrich_coefficients__coeff_1__m_3__r_30',
    'ACC__friedrich_coefficients__coeff_2__m_3__r_30',
    'ACC__friedrich_coefficients__coeff_3__m_3__r_30',
    'ACC__max_langevin_fixed_point__m_3__r_30',
    'ACC__query_similarity_count__query_None__threshold_0.0'
]

x = x.drop(columns=columns_drop)


from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy='mean')
x=pd.DataFrame(imputer.fit_transform(x),columns=x.columns)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x, y)

sfm = SelectFromModel(rf, prefit=True)
X_selected = sfm.transform(x)

selected_feature_indices = sfm.get_support(indices=True)
selected_feature_names = x.columns[selected_feature_indices]

print(f"Original number of features: {x.shape[1]}")
print(f"Number of features after selection: {X_selected.shape[1]}")
print("Selected features:", list(selected_feature_names))

Original number of features: 807
Number of features after selection: 284
Selected features: ['ADD', 'WURS', 'ASRS', 'MADRS', 'HADS_D', 'ACC__mean_abs_change', 'ACC__length', 'ACC__standard_deviation', 'ACC__skewness', 'ACC__root_mean_square', 'ACC__absolute_sum_of_changes', 'ACC__count_below_mean', 'ACC__last_location_of_maximum', 'ACC__percentage_of_reoccurring_values_to_all_values', 'ACC__sum_of_reoccurring_data_points', 'ACC__ratio_value_number_to_time_series_length', 'ACC__sample_entropy', 'ACC__benford_correlation', 'ACC__time_reversal_asymmetry_statistic__lag_1', 'ACC__c3__lag_2', 'ACC__cid_ce__normalize_False', 'ACC__quantile__q_0.4', 'ACC__autocorrelation__lag_5', 'ACC__partial_autocorrelation__lag_4', 'ACC__partial_autocorrelation__lag_5', 'ACC__partial_autocorrelation__lag_7', 'ACC__partial_autocorrelation__lag_8', 'ACC__partial_autocorrelation__lag_9', 'ACC__number_cwt_peaks__n_5', 'ACC__number_peaks__n_1', 'ACC__number_peaks__n_5', 'ACC__number_peaks__n_10', 'ACC__number_pe

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)

In [25]:
print(x_test[0:1])

[[-6.29723530e-01  0.00000000e+00  0.00000000e+00  2.40595393e-16
   0.00000000e+00  3.80124159e-01  1.37332058e+00 -3.62885763e-01
  -2.86346244e-01 -2.07668584e-01  1.19387248e+00  8.52852376e-01
   9.39133680e-02  5.45268143e-01  7.52226690e-01 -6.68307344e-01
   1.16425094e+00  2.98638228e-01 -3.64846896e-01 -5.28147865e-01
   4.53294798e-01  1.12937400e+00 -3.45800562e-01  7.46311715e-01
   9.93774499e-01  3.46129081e-01  1.42640284e+00  3.15518987e-01
   1.95154688e+00  2.02641711e+00  1.47079373e+00  1.39855057e+00
   1.21827506e+00  7.29261303e-02  1.89479273e-01 -5.70607351e-01
  -2.00899413e-01 -4.20832601e-01 -5.53627735e-01 -3.82020453e-01
  -5.97393284e-01 -1.02155107e+00 -8.41246142e-01 -5.22451256e-01
  -8.28670108e-01  2.66522635e-01 -2.87593082e-01 -6.63099514e-01
  -2.44315348e-01  5.10136218e-01 -1.98431903e-01 -2.56396125e-01
  -1.49878193e-01 -8.08391399e-01  4.33171116e-01  7.19382562e-01
  -2.46321700e-02 -1.35625246e+00 -6.62870345e-01 -1.08987167e+00
  -2.12415

In [15]:
from xgboost import XGBClassifier

gradient=XGBClassifier()
gradient.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [16]:
from sklearn.metrics import classification_report,accuracy_score
y_pred=gradient.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.90      1.00      0.95         9

    accuracy                           0.94        17
   macro avg       0.95      0.94      0.94        17
weighted avg       0.95      0.94      0.94        17



In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Define the XGBoost model
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of boosting rounds
    'learning_rate': [0.01, 0.1, 0.2],  # Step size shrinkage
    'max_depth': [3, 5, 7],  # Maximum depth of a tree
    'subsample': [0.8, 1.0],  # Subsample ratio of the training instance
    'colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2]  # Minimum loss reduction required to make a further partition on a leaf node
}

# Define the cross-validation strategy
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=xgb,
                           param_grid=param_grid,
                           cv=cv_strategy,
                           scoring='accuracy',
                           verbose=2,
                           n_jobs=-1)

# Ensure x_train and y_train are ready
# (Assume x_train and y_train are already defined and preprocessed)

print("Starting GridSearchCV for XGBoost...")
grid_search.fit(x_train, y_train)
print("GridSearchCV complete.")

# Print the best hyperparameters and scores
print("\nBest hyperparameters found:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Retrieve the best model
best_xgb_model = grid_search.best_estimator_

# Evaluate the best model on the test set
print("\nEvaluating the best model on the test set...")
y_pred_best = best_xgb_model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_pred_best)
print(f"Test Set Accuracy with best XGBoost model: {test_accuracy:.4f}")
print("\nClassification Report for the best XGBoost model on Test Set:")
print(classification_report(y_test, y_pred_best))

Starting GridSearchCV for XGBoost...
Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [17:55:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV complete.

Best hyperparameters found: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best cross-validation accuracy: 0.8294871794871795

Evaluating the best model on the test set...
Test Set Accuracy with best XGBoost model: 0.9412

Classification Report for the best XGBoost model on Test Set:
              precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.90      1.00      0.95         9

    accuracy                           0.94        17
   macro avg       0.95      0.94      0.94        17
weighted avg       0.95      0.94      0.94        17



In [ ]:
df['ADHD'].value_counts()

,count
ADHD,
0,52
1,51


In [21]:
import pickle
with open("adhdmodel.pkl","wb") as file:
  pickle.dump(gradient,file)